In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("241210_02_SparkSQL").getOrCreate()
spark

24/12/10 14:50:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 14:50:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [14]:
# 파일 2개 합치기

# emp.csv 파일 불러오기 
df = spark.read.format("csv")\
    .option("header",'true')\
    .option('inferSchema', 'true')\
    .load("data/emp.csv")

# dept.csv 파일 불러오기 
dept_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/dept.csv')

In [15]:
df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: string (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [16]:
df.show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
| 9292|  JACK|

In [17]:
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [18]:
dept_df.show()

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [9]:
# 각 파일, 데이터프레임의 VIEW 만들기 
df.createOrReplaceTempView("emp_df_view")
dept_df.createOrReplaceTempView("dept_df_view")

In [10]:
# 뷰에서 조인 

join_query = '''
SELECT *
FROM emp_df_view JOIN dept_df_view ON emp_df_view.deptno = dept_df_view.deptno
'''

viewEmpDept = spark.sql(join_query)
viewEmpDept.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|    10|A

In [13]:
# 조인된 데이터프레임 테이블 VIEW만들기
viewEmpDept.createOrReplaceTempView("viewEmpDept")

# LOC 이 NEW YORK 인 곳 필터링 
v_query = '''
select *
from viewEmpDept
where loc = "NEW YORK"
'''

#case1
spark.sql(v_query).show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|    10|ACCOUNTING|NEW YORK|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|    10|ACCOUNTING|NEW YORK|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|    10|ACCOUNTING|NEW YORK|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+



In [ ]:
v_query = '''
select *
from viewEmpDept
where loc = "NEW YORK"
'''
spark.sql(v_query).show()

In [21]:
# subquery

query2 = '''
select *
from emp_df_view
where deptno = (
    select deptno
    from dept_df_view
    where loc = 'NEW YORK'
)
'''

#case 2 
spark.sql(query2).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
+-----+------+---------+----+----------+----+----+------+



In [22]:
#case1의 실행 계획 - join
spark.sql(v_query).explain()

== Physical Plan ==
*(2) BroadcastHashJoin [deptno#23], [deptno#111], Inner, BuildRight, false
:- *(2) Filter isnotnull(deptno#23)
:  +- FileScan csv [empno#16,ename#17,job#18,mgr#19,hiredate#20,sal#21,comm#22,deptno#23] Batched: false, DataFilters: [isnotnull(deptno#23)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab05/src/data/emp.csv], PartitionFilters: [], PushedFilters: [IsNotNull(deptno)], ReadSchema: struct<empno:int,ename:string,job:string,mgr:int,hiredate:string,sal:int,comm:int,deptno:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#315]
   +- *(1) Filter ((isnotnull(loc#113) AND (loc#113 = NEW YORK)) AND isnotnull(deptno#111))
      +- FileScan csv [deptno#111,dname#112,loc#113] Batched: false, DataFilters: [isnotnull(loc#113), (loc#113 = NEW YORK), isnotnull(deptno#111)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab05/src/data/dept.csv], PartitionFilters: [], PushedFilters: [IsNotNull(loc), Eq

In [23]:
#case 2 
spark.sql(query2).explain()

== Physical Plan ==
*(1) Filter (isnotnull(deptno#23) AND (deptno#23 = Subquery scalar-subquery#478, [id=#334]))
:  +- Subquery scalar-subquery#478, [id=#334]
:     +- *(1) Project [deptno#111]
:        +- *(1) Filter (isnotnull(loc#113) AND (loc#113 = NEW YORK))
:           +- FileScan csv [deptno#111,loc#113] Batched: false, DataFilters: [isnotnull(loc#113), (loc#113 = NEW YORK)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab05/src/data/dept.csv], PartitionFilters: [], PushedFilters: [IsNotNull(loc), EqualTo(loc,NEW YORK)], ReadSchema: struct<deptno:int,loc:string>
+- FileScan csv [empno#16,ename#17,job#18,mgr#19,hiredate#20,sal#21,comm#22,deptno#23] Batched: false, DataFilters: [isnotnull(deptno#23)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab05/src/data/emp.csv], PartitionFilters: [], PushedFilters: [IsNotNull(deptno)], ReadSchema: struct<empno:int,ename:string,job:string,mgr:int,hiredate:string,sal:int,comm:int,deptno:int>




In [24]:
spark.stop()